<a href="https://colab.research.google.com/github/rishabh135/2015/blob/master/all_financials_financialdatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi there 👋 - [financialdatasets.ai](https://www.financialdatasets.ai) is a stock market API that provides:

*   income statements
*   balance sheets
*   cash flow statements
*   sec filings
*   stock prices

This is a quick start guide on how to get **all financial statements** for a company in a single API call.

The data comes from the `financials/` endpoint.

**Questions?**

The full API documentation is available [here](https://docs.financialdatasets.ai/api-reference/endpoint/financials/all-financial-statements).

You can also DM me on [twitter](https://www.x.com/virattt).

# Call the financials API

In [ ]:
import requests

ticker = 'AAPL'
period = 'annual' # possible values are 'annual', 'quarterly', or 'ttm'
limit = 10        # the number of statements to return

url = (
    f'https://api.financialdatasets.ai/financials/'
    f'?ticker={ticker}'
    f'&period={period}'
    f'&limit={limit}'
)

response = requests.get(url)

# Parse the response

In [ ]:
# parse financials from the response
financials = response.json().get('financials')

# get income statements
income_statements = financials.get('income_statements')

# get balance sheets
balance_sheets = financials.get('balance_sheets')

# get cash flow statements
cash_flow_statements = financials.get('cash_flow_statements')

In [ ]:
# Pretty print the response JSON
import json
print(json.dumps(financials, indent=2))

# Visualize data in table

In [ ]:
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Helper function to format snakecase to capitalized words
def format_column_name(name):
    return ' '.join(word.capitalize() for word in name.split('_'))

# Function to create a DataFrame from a list of dictionaries
def create_dataframe(data_list):
    df = pd.DataFrame(data_list)
    df.set_index('report_period', inplace=True)
    df = df.drop(['ticker', 'calendar_date', 'period'], axis=1)
    df = df.T
    df.index = df.index.map(format_column_name)  # Apply formatting to index
    return df

# Create DataFrames
income_statements_df = create_dataframe(income_statements)
balance_sheets_df = create_dataframe(balance_sheets)
cash_flow_statements_df = create_dataframe(cash_flow_statements)

# Function to create a styled DataFrame
def style_dataframe(df):
    return df.style.format("${:,.0f}").set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold')]},
        {'selector': 'td', 'props': [('text-align', 'right')]},
        {'selector': 'th.row_heading', 'props': [('text-align', 'left')]},  # Left-align first column
        {'selector': 'td:first-child', 'props': [('text-align', 'left')]}  # Left-align first column
    ]).set_properties(**{'min-width': '120px', 'font-size': '10pt'})  # Increased cell width

# Create styled DataFrames
income_styled = style_dataframe(income_statements_df)
balance_styled = style_dataframe(balance_sheets_df)
cash_flow_styled = style_dataframe(cash_flow_statements_df)

# Create tab contents
tab_contents = [
    widgets.HTML(income_styled.to_html()),
    widgets.HTML(balance_styled.to_html()),
    widgets.HTML(cash_flow_styled.to_html())
]

# Create tabs
tab = widgets.Tab()
tab.children = tab_contents
tab.set_title(0, 'Income Statements')
tab.set_title(1, 'Balance Sheets')
tab.set_title(2, 'Cash Flow Statements')

# Display the tabs
display(tab)

# Add CSS for better table display
display(HTML("""
<style>
.widget-tab-contents {
    overflow-x: auto;
    max-width: 100%;
}
table {
    white-space: nowrap;
}
th.row_heading, td:first-child {
    position: sticky;
    left: 0;
    background-color: white;
    z-index: 1;
}
th.row_heading {
    background-color: #f2f2f2;
}
</style>
"""))